In [2]:
# Imports -  These are all you need for the assignment: do not import additional packages
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import requests
#from geopy.geocoders import Nominatim
from urllib.request import Request
import math

from pandas.core import datetools
import patsy
import statsmodels.api as sm
import scipy.stats as stats
from scipy.stats import ttest_ind, chisquare, normaltest


/home/nhhuang/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  del sys.path[0]


In [3]:
#load Los Angeles Data
df_LosAngeles = pd.read_csv("data/Crime_Data_from_2010_to_Present.csv") 

#Remove unneeded columns
df_LosAngeles_Trimmed = df_LosAngeles[['Date Occurred','Time Occurred','Crime Code','Location ']]

In [6]:
#Load Median Housing Data
df_HousePrice = pd.read_csv("data/Zip_MedianListingPrice_AllHomes.csv")

#Filter by Los Angeles Only
df_LAHousePrice = df_HousePrice[df_HousePrice['City'] == 'Los Angeles']

df_LAHousePrice.head()

,RegionName,City,State,Metro,CountyName,SizeRank,2010-01,2010-02,2010-03,2010-04,...,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01
34,90046,Los Angeles,CA,Los Angeles-Long Beach-Anaheim,Los Angeles,35,NaN,NaN,NaN,NaN,...,1399000.0,1475000.0,1450000.0,1487500.0,1399000.0,1400900.0,1635000.0,1592500.0,1699000.0,1679000.0
88,90044,Los Angeles,CA,Los Angeles-Long Beach-Anaheim,Los Angeles,89,NaN,NaN,NaN,NaN,...,359900.0,369900.0,377000.0,389999.0,389999.0,399900.0,405000.0,410000.0,419000.0,419900.0
108,90026,Los Angeles,CA,Los Angeles-Long Beach-Anaheim,Los Angeles,109,NaN,NaN,NaN,NaN,...,883500.0,890000.0,889000.0,899000.0,899000.0,944000.0,989000.0,939000.0,944000.0,899990.0
139,90066,Los Angeles,CA,Los Angeles-Long Beach-Anaheim,Los Angeles,140,NaN,NaN,NaN,NaN,...,1349000.0,1325000.0,1385000.0,1349000.0,1315000.0,1349499.5,1349000.0,1357500.0,1399000.0,1399000.0
168,90019,Los Angeles,CA,Los Angeles-Long Beach-Anaheim,Los Angeles,169,NaN,NaN,NaN,NaN,...,993000.0,1033000.0,1000000.0,1070000.0,1095000.0,1125000.0,1150000.0,1190000.0,1150000.0,1150000.0


In [21]:
#Load US Zip Codes
df_US_Zip = pd.read_csv("data/US.txt",header = None, sep = '\t')
df_US_Zip.columns = ['Country', 'ZipCode','City','State','Stabv','ignore1','ignore2','ignore3', 'ignore4','long','lat','ignore5']

#Sort Only Los Angeles
df_US_Zip_trimmed = df_US_Zip[df_US_Zip['City'] == 'Los Angeles']

df_US_Zip_trimmed = df_US_Zip_trimmed[['Country','ZipCode','City','State','long','lat']]

df_US_Zip_trimmed.head()

,Country,ZipCode,City,State,long,lat
2811,US,90001,Los Angeles,California,33.9731,-118.2479
2812,US,90002,Los Angeles,California,33.9497,-118.2462
2813,US,90003,Los Angeles,California,33.9653,-118.2727
2814,US,90004,Los Angeles,California,34.0762,-118.3029
2815,US,90005,Los Angeles,California,34.0585,-118.3012


In [45]:
def euclidean(x1, y1, x2, y2):
    return math.sqrt((x1-x2)**2 + (y1-y2)**2)
#Function to get closest zip code
def get_closest_zip(longlat):
    distance = 100
    zipcode = 'NA'
    try:
        [long,lat] = str(longlat).split(',')
        for index, row in df_US_Zip_trimmed.iterrows():
            currDistance = (euclidean(float(long),float(lat),float(row['long']),float(row['lat'])))
            if(float(currDistance) <= float(distance) ):
                distance = currDistance
                zipcode = row['ZipCode']
    except (RuntimeError, TypeError, NameError, ValueError):
        zipcode = 'NA'
        
    return zipcode

get_closest_zip('test')

'NA'

In [24]:
#locator hack

def get_geolocator():
    geolocator = Nominatim()

    requester = geolocator.urlopen

    def requester_hack(req, **kwargs):
        req = Request(url=req, headers=geolocator.headers)
        return requester(req, **kwargs)

    geolocator.urlopen = requester_hack

    return geolocator

In [54]:
#Convert location to address
location = get_geolocator().reverse('34.0454, -118.3157')
address = location.address
print(location)


1429, South Van Ness Avenue, Country Club Park, Cienega, LA, Los Angeles County, California, 90019, United States of America


In [8]:
df_LosAngeles_Trimmed.head()
df_LosAngeles.shape

(1692056, 26)

In [11]:
#Sort los Angeles data by 
df_LosAngeles_Trimmed = df_LosAngeles_Trimmed.sort_values(by='Date Occurred')

In [12]:
# Removes the paranthesis from locations
df_LosAngeles_Trimmed['Location ']= df_LosAngeles_Trimmed['Location '].str.replace(r"\(","")
df_LosAngeles_Trimmed['Location ']= df_LosAngeles_Trimmed['Location '].str.replace(r"\)","")


In [13]:
df_LosAngeles_Trimmed.head(10)

,Date Occurred,Time Occurred,Crime Code,Location
254354,01/01/2010,1200,740,"34.1103, -118.3333"
319962,01/01/2010,2300,812,"33.795, -118.2711"
293441,01/01/2010,800,810,"33.9637, -118.3003"
319964,01/01/2010,1200,354,"33.7742, -118.262"
309604,01/01/2010,1,354,"34.2354, -118.4502"
319965,01/01/2010,600,354,"33.7569, -118.3026"
319966,01/01/2010,800,354,"33.7915, -118.2939"
319967,01/01/2010,1,354,"33.7951, -118.2752"
319969,01/01/2010,1,354,"33.7831, -118.2552"
138172,01/01/2010,1800,510,"34.2891, -118.4135"


In [40]:
#Split the long and lat from the coordinates
df_LosAngeles_Trimmed_Location = df_LosAngeles_Trimmed['Location '].str.split(',', expand=True)

df_LosAngeles_Trimmed_Location.columns = ['Long', 'Lat']

df_LosAngeles_Trimmed_LongLat = pd.concat([df_LosAngeles_Trimmed,df_LosAngeles_Trimmed_Location],axis=1)

df_LosAngeles_Trimmed_LongLat_droppedNA = df_LosAngeles_Trimmed_LongLat.dropna(how = 'any')

(df_LosAngeles_Trimmed_LongLat_droppedNA['Lat']==0).sum()

0

In [77]:
#Use service to get zip, too much data doesn't work D:

#df_LosAngeles_Trimmed_test.assign(Address=df_LosAngeles_Trimmed_test['Location '].apply(get_geolocator().reverse))

In [46]:
#get zip code

df_LosAngeles_Trimmed_LongLat_droppedNA = df_LosAngeles_Trimmed_LongLat_droppedNA.assign(ZipCode=df_LosAngeles_Trimmed_LongLat['Location '].apply(get_closest_zip))
df_LosAngeles_Trimmed_LongLat_droppedNA.head()

,Date Occurred,Time Occurred,Crime Code,Location,Long,Lat,ZipCode
254354,01/01/2010,1200,740,"34.1103, -118.3333",34.1103,-118.3333,90068
319962,01/01/2010,2300,812,"33.795, -118.2711",33.795,-118.2711,90095
293441,01/01/2010,800,810,"33.9637, -118.3003",33.9637,-118.3003,90047
319964,01/01/2010,1200,354,"33.7742, -118.262",33.7742,-118.262,90095
309604,01/01/2010,1,354,"34.2354, -118.4502",34.2354,-118.4502,90077


In [ ]:
df['date'] = pd.to_datetime(df['date'])
df['year'], df['month'] = df['date'].dt.year, df['date'].dt.month

In [47]:
df_LosAngeles_Trimmed_LongLat_droppedNA.to_csv('LADataWithClosestZip')